In [1]:
import sys
if (path := "C:/Users/Tom/PycharmProjects/python-hvac") not in sys.path:
    sys.path.append(path)

In [2]:
from hvac import Quantity, print_doc_string

Q_ = Quantity

# Chapter 1: Construction Assembly of a Vertical Exterior Wall
---

A construction assembly is composed of construction layers, ordered from the exterior side towards the interior side of the wall. There are different kinds of construction layers. On the exterior surface side and on the interior surface side of a wall, there is always a surface film, where heat is transferred to or from the wall by combined convection and radiation. The wall itself is internally composed of successive solid material layers, and it may also contain an air layer. In this notebook the basics of defining the construction assembly of a vertical exterior masonry wall are demonstrated.

## Surface Film

To create a surface film, class method `create(...)` of the `SurfaceFilm` class needs to be called.

In [3]:
from hvac.cooling_load_calc import SurfaceFilm, Geometry, HeatFlowDirection

In [4]:
print_doc_string(SurfaceFilm.create)

Creates a `SurfaceFilm` object.

Parameters
----------
ID: str
    Name to identify the surface film.
geometry: Geometry
    `Geometry` object with the area (A) of the surface film.
    Note: in case of a surface film, the film thickness is not 
    considered.
heat_flow_dir: HeatFlowDirection
    `HeatFlowDirection` object indicating the direction of the heat flow
    perpendicular to the surface. Can be HORIZONTAL in case of a vertical
    surface, and UPWARDS or DOWNWARDS in case of a horizontal surface.
T_mn: Quantity
    The mean thermodynamic temperature of the surface and of its
    surroundings (e.g. operative temperature for internal surfaces,
    external air temperature or sol-air temperature for external 
    surfaces).
surf_emissivity: Quantity, default 0.9
    The hemispherical emissivity of the surface.
is_internal_surf: bool, default True
    Indicates whether the surface is internal or external.
wind_speed: Quantity, default 4 m/s
    Wind speed. Only used with externa

### Exterior Surface Film

In [5]:
T_ext = Q_(-8, 'degC')  # exterior air temperature

In [6]:
ext_film = SurfaceFilm.create(
    ID='ext-surf-film',
    geometry=Geometry(),
    heat_flow_dir=HeatFlowDirection.HORIZONTAL,
    T_mn=T_ext,
    is_internal_surf=False,
    wind_speed=Q_(4, 'm / s')
)

> **Notes**
> - An object of class `Geometry` contains the area `A` and the thickness `t` of a layer. Default values are: `A` = 1 m² and `t` = 0 m. So, the exterior surface film has an area of 1 m² and zero thickness.
> - As the wall is vertical, the heat flows in a horizontal direction through the surface film.
> - `is_internal_surf=False` indicates that the surface film is on the exterior side of the wall. 

### Interior Surface Film

In [7]:
T_zone = Q_(20, 'degC')  # zone air temperature

In [8]:
int_film = SurfaceFilm.create(
    ID='int-surf-film',
    geometry=Geometry(),
    heat_flow_dir=HeatFlowDirection.HORIZONTAL,
    T_mn=T_zone
)

> **Notes**
> - By default, a `SurfaceFilm` object is assumed to be situated on the interior side of a construction assembly.

## Solid Layers

In [9]:
from hvac.cooling_load_calc import SolidLayer, Material

In [10]:
print_doc_string(SolidLayer.create)

Creates a `SolidLayer` object.

Parameters
----------
ID:
    Name to identify the layer in the construction assembly.
geometry:
    `Geometry` object that holds the surface area (`A`) and thickness
    (`t`) of the layer.
material:
    `Material` object that holds the material properties of the layer
    (`k`, `rho`, `c`, `R`) where `k` is the thermal conductivity of
    the material, `rho` is the mass density, `c` is the specific heat,
    and `R` is the unit thermal resistance (default None).
num_slices:
    A solid layer can be subdivided into a number of slices. Each slice
    will correspond with a temperature node in the linear thermal 
    network model of an exterior building element. 

Parameter Types & Defaults
--------------------------
ID: 'str'
geometry: 'Geometry'
material: 'Material'
num_slices: 'int' = 1


In [11]:
outer_leaf = SolidLayer.create(
    ID='brick-wall-15',
    geometry=Geometry(t=Q_(15, 'cm')), # layer thickness
    material=Material(
        k=Q_(1.59, 'W / (m * K)'),  # conductivity of material
        rho=Q_(1500, 'kg / m**3'),  # mass density of material
        c=Q_(840, 'J / (kg * K)')   # specific heat of material
    ),
    num_slices=5  # the outer leaf is subdivided into 5 slices having a thickness of 3 cm
)

In [12]:
inner_leaf = SolidLayer.create(
    ID='brick-wall-18',
    geometry=Geometry(t=Q_(18, 'cm')),
    material=Material(
        k=Q_(1.59, 'W / (m * K)'),
        rho=Q_(1500, 'kg / m**3'),
        c=Q_(840, 'J / (kg * K)')
    ),
    num_slices=6
)

In [13]:
gypsum_layer = SolidLayer.create(
    ID='gypsum',
    geometry=Geometry(t=Q_(1.5, 'cm')),
    material=Material(
        k=Q_(0.56, 'W / (m * K)'),
        rho=Q_(1300, 'kg / m**3'),
        c=Q_(840, 'J / (kg * K)')
    )
)

## Air Layer

Between the outer leaf and the inner leaf of the vertical wall is an air layer.

Before we can define the air layer, we need to know the temperature difference between the outer and the inner side of the air layer and the average temperature of the air layer. For this, we use the `AirLayerTemperatureSolver` class. When instantiating the solver, we need to pass it:
- the temperature `T_ext` on the exterior side of the wall
- the temperature `T_int` on the interior side of the wall
- the thermal resistance `R_ea` between the exterior side of the wall and the exterior side of the air layer inside the wall.
- the thermal resistance `R_ai` between the interior side of the air layer inside the wall and the interior side of the wall.

When the solver is instantiated, we can call its `solve()` method. This method uses Scipy's `minimize()` function to find the temperature on both the exterior and the interior side of the air layer by balancing the heat flow from the exterior side of the wall to the exterior side of the air layer and the heat flow from the interior side of the air layer to the interior side of the wall (at steady-state these heat flows should be equal). The method needs intial guesses for the temperatures on the exterior and on the interior side of the air layer. The method returns (1) the temperature on the exterior side of the air layer, (2) the temperature on the interior side of the air layer, (3) the absolute value of the temperature difference between the exterior and the interior side of the air layer, and (4) the average temperature of the air layer. We only need the temperature difference and the average or mean temperature to define the air layer.

In [14]:
from hvac.cooling_load_calc import AirLayerTemperatureSolver

In [15]:
solver = AirLayerTemperatureSolver(
    T_ext=T_ext,
    T_int=T_zone,
    R_ea=ext_film.R + outer_leaf.R,
    R_ai=inner_leaf.R + gypsum_layer.R + int_film.R
)
*_, dT, T_mn = solver.solve(
    T_ae_guess=T_ext.to('K') - Q_(3, 'K'),
    T_ai_guess=T_zone.to('K') + Q_(5, 'K')
)

In [16]:
from hvac.cooling_load_calc import AirLayer

In [17]:
print_doc_string(AirLayer.create)

Creates an `AirLayer` object.

Parameters
----------
ID: str
    Name to identify the airspace in a building component.
geometry: Geometry
    Note: the maximum allowable thickness of the airspace is 0.3 m.
    The thickness is the dimension in the direction of heat flow.
dT: Quantity
    The temperature difference across the airspace.
heat_flow_dir: HeatFlowDirection
    Enum-type object indicating the direction of the heat flow
    perpendicular to the surface. Can be HORIZONTAL in case of a vertical
    surface, and UPWARDS or DOWNWARDS in case of a horizontal surface.
T_mn: Quantity
    The mean thermodynamic temperature of the surfaces and the airspace.
surf_emissivities: Tuple[Quantity, Quantity], (Q_(0.9, 'frac'), Q_(0.9, 'frac'))
    Emissivity of surfaces on both sides of the airspace.
angle: Quantity, optional
    Inclination angle of the airspace with respect to the horizontal.
    Value can be between 0° and 90° included.

Parameter Types & Defaults
------------------------

In [18]:
air_layer = AirLayer.create(
    ID='air-layer',
    geometry=Geometry(t=Q_(5, 'cm'), w=Q_(float('inf'), 'm')),
    dT=dT,
    heat_flow_dir=HeatFlowDirection.HORIZONTAL,
    T_mn=T_mn
)

> **Notes**
> - Instead of assigning a value to the area `A` directly, it is also possible to set the width `w` and/or the height `h` of a layer separately. Here we set the width of the air layer to infinity, to distinguish it from an air void, which is a specific kind of air layer having a width that is less than 10 times the air layer thickness.

## Construction Assembly

Once we have all the construction layers the construction assembly is composed of, we can put them together in a `ConstructionAssembly` object.

In [19]:
from hvac.cooling_load_calc import ConstructionAssembly

In [20]:
print_doc_string(ConstructionAssembly.create)

Creates a `ConstructionAssembly` object.

Parameters
----------
ID: str
    Name to identify the construction assembly.
layers: list of `ConstructionLayer` objects, optional, default None
    A construction assembly is thought to be composed of successive
    construction layers, arranged from the outdoor to the indoor
    environment.
    Note that it is expected that a construction assembly always has an
    exterior and an interior surface film.
U: Quantity, default None
    U-value of the construction assembly per unit area. Can be used if
    the layered composition of the construction assembly is unknown.
R: Quantity, default None
    R-value of the construction assembly per unit area. Can be used if
    the layered composition of the construction assembly is unknown.
geometry: Geometry, default None
    The geometrical properties of the construction assembly (area and
    thickness). Use this if `layers` is None, and `U` or `R` are used
    instead.

Parameter Types & Defaults
-

In [21]:
constr_assem = ConstructionAssembly.create(
    ID='exterior-wall',
    layers=[
        ext_film,
        outer_leaf,
        air_layer,
        inner_leaf,
        gypsum_layer,
        int_film
    ]
)

> **Notes**
> - The layers must always be ordered from the exterior towards the interior side of the construction assembly.

### Unit Thermal Resistance and Conductance of the Construction Assembly

In [22]:
print(f"{constr_assem.R.to('K * m**2 / W'):~P.3f}")

0.561 K·m²/W


In [23]:
print(f"{constr_assem.U.to('W / (K * m**2)'):~P.3f}")

1.782 W/K/m²


### Save the Construction Assembly on a Shelf

First, we need to set the file path to the shelf. (Here we have manually created in advance a subdirectory `construction_shelf` in the same directory of this notebook.)

In [24]:
ConstructionAssembly.db_path = "./construction_shelf/assemblies"

Then call `save()` on the object.

In [25]:
constr_assem.save()